## Set up environment

In [3]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    """Set seed for reproducibility"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"✅ Seed set to {seed} for future runs")

set_seed(42)

✅ Seed set to 42 for future runs


In [4]:
# Pull OpenGait, GETA and Custom files from github
!pwd
!rm -rf GAITBASE_x_GETA #!! careful in removing may contains model weights 
!git clone https://github.com/eli-bigman/GAITBASE_x_GETA.git

/kaggle/working
Cloning into 'GAITBASE_x_GETA'...
remote: Enumerating objects: 885, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 885 (delta 26), reused 23 (delta 14), pack-reused 843 (from 1)
Receiving objects: 100% (885/885), 20.11 MiB | 4.64 MiB/s, done.
Resolving deltas: 100% (432/432), done.


In [21]:
#Geta requirements 
!pip install "torch==2.0.1+cu117" \
             "torchvision==0.15.2+cu117" \
             "torchaudio==2.0.2" \
             --extra-index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 474.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 98.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 81.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 28.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled t

In [48]:
#OpenGait requirements
# !pip install tqdm pyyaml tensorboard opencv-python kornia einops # installed by defualt
# !pip install torch==1.10 torchvision==0.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 896.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

## Prepare dataset- Casia B - Merge the frams in the dataset into pkl format (run only once and persist)

In [6]:
from pathlib import Path
src = Path("/kaggle/input/")

if src.exists():
    print("Input directory is exist")
else:
    print("Dataset dir missing")

Input directory is exist


### Copy dataset into working dir

In [10]:
# !rm -rf /kaggle/working/casia-b

In [11]:
# !rsync -aq --progress /kaggle/input/ /kaggle/working/casia-b

### Apply Gaitbase pretreatment to dataset

In [12]:
# !python3 GAITBASE_x_GETA/OpenGait/datasets/pretreatment.py --input_path "/kaggle/working/casia-b/output" --output_path "/kaggle/working/CASIA-B-pkl"

Pretreating: 100%|████████████████████| 13592/13592 [06:04<00:00, 37.26folder/s]


## Training the GAITBASE with GETA on CASIA B

### Update dataset path in yaml file

In [18]:
#on model resumption copy weights

# Create the directory structure OpenGait expects
# !mkdir -p output/CASIA-B/Baseline/GaitBase_GETA_40K_Production/checkpoints

# Copy your checkpoint to the expected location
# !cp -r "./checkpoints" "output/CASIA-B/Baseline/GaitBase_GETA_40K_Production/"

In [6]:
# test GETA_x_OpenGait Setup
!python /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/test_setup.py

=== GETA + OpenGait Setup Test ===
✅ GETA path found: /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/../../geta
✅ GETA import successful
✅ OpenGait path found: /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/../../OpenGait
2025-08-06 17:23:16.900387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754500997.122305      98 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754500997.189181      98 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
✅ OpenGait import successful
✅ Default config found: /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/../../OpenGait/configs/default.yaml
✅ Config loading successful
✅ CUDA

In [ ]:
!python /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/train_gaitbase_with_geta.py \
    --config /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/gaitbase_geta.yaml \
    --gpu 0 \
    --validate 

✅ Added GETA path: /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/../../geta
✅ Added OpenGait path: /kaggle/working/GAITBASE_x_GETA/gaitbase_compression_GETA/src/../../OpenGait
✅ Found GETA at: /kaggle/working/GAITBASE_x_GETA/geta
✅ Found OpenGait at: /kaggle/working/GAITBASE_x_GETA/OpenGait
2025-08-06 17:23:44.249540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754501024.270799     117 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754501024.277288     117 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
✅ Initialized distributed training with nccl
📁 Changed working directory to: /kaggle/working/GAITBASE_x_GETA/OpenGait
🔧 Fixed dat

In [ ]:
# import shutil

# shutil.copytree('/kaggle/working/GAITBASE_x_GETA/OpenGait/output', '/kaggle/working/models_output')


In [15]:
#util function to print dir tree
import os

def print_tree(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = '│   ' * level + '├── '
        print(f"{indent}{os.path.basename(root)}/")
        subindent = '│   ' * (level + 1) + '├── '
        for f in files:
            print(f"{subindent}{f}")

# print_tree('/kaggle/working/GAITBASE_x_GETA/OpenGait')
# print_tree('/kaggle/working/checkpoints')



In [16]:
ls

CASIA-B-pkl/  GAITBASE_x_GETA/      log_vanilla.txt
checkpoints/  intermediate_models/


In [18]:
print_tree('/kaggle/working/checkpoints/')

├── /
│   ├── GaitBase_GETA_60K_Production-04000.pt
│   ├── GaitBase_GETA_60K_Production-18000.pt
│   ├── GaitBase_GETA_60K_Production-12000.pt
│   ├── GaitBase_GETA_60K_Production-26000.pt
│   ├── GaitBase_GETA_60K_Production-02000.pt
│   ├── GaitBase_GETA_60K_Production-38000.pt
│   ├── GaitBase_GETA_60K_Production-08000.pt
│   ├── GaitBase_GETA_60K_Production-48000.pt
│   ├── GaitBase_GETA_60K_Production-36000.pt
│   ├── GaitBase_GETA_60K_Production-10000.pt
│   ├── GaitBase_GETA_60K_Production-42000.pt
│   ├── GaitBase_GETA_60K_Production-28000.pt
│   ├── GaitBase_GETA_60K_Production-44000.pt
│   ├── GaitBase_GETA_60K_Production-34000.pt
│   ├── GaitBase_GETA_60K_Production-40000.pt
│   ├── GaitBase_GETA_60K_Production-30000.pt
│   ├── GaitBase_GETA_60K_Production-20000.pt
│   ├── GaitBase_GETA_60K_Production-22000.pt
│   ├── GaitBase_GETA_60K_Production-14000.pt
│   ├── GaitBase_GETA_60K_Production-06000.pt
│   ├── GaitBase_GETA_60K_Production-46000.pt
│   ├── GaitBase_GETA_60K_Pr

## Test the GETA compressed Gaitbase model

In [ ]:

!python -m torch.distributed.launch --nproc_per_node=2 /kaggle/working/GAITBASE_x_GETA/OpenGait/opengait/main.py --cfgs ./configs/gaitbase/gaitbase_da_casiab.yaml --phase test


## Analyze the compressed GaitBase model